# Notebook content overview

This notebook ocntains the application of the linear transformation required to extract the layer to which each functionally matched cell belongs to.

    NOTE: In this notebook the file clean_functional_neurons.pkl is saved
    NOTE: In this notebook func_neuronsl234.csv is saved containing only neurons from layer 2/3 and layer 4

# Imports

In [1]:
import random
import datajoint as dj
from phase3 import nda, func, utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
from collections import defaultdict
from PIL import Image
import warnings
import pickle
import os
from caveclient import CAVEclient
import nglui
!pip install standard-transform
from standard_transform import minnie_transform_vx

Connecting microns@db.datajoint.com:3306
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### CAVE client set up

In [2]:
client = CAVEclient()
auth = client.auth
auth.get_new_token()

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.daf-apis.com/auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


In [1]:
#This should be replaced with the token you see after you visit the website.
new_token = ''
auth.save_token(token=new_token)

In [5]:
#Connect to CaveClient
client = CAVEclient('minnie65_public_v343')

In [6]:
func_data=client.materialize.query_table('functional_coreg')

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


# Extracting layers of functional cells

In [7]:
#Define transform and import CAVE data on functionally matched cells
tform_vx = minnie_transform_vx()
func_data=client.materialize.query_table('functional_coreg')

#Find the area of the cortex the cells belong to (V1 or HVA)
cortex_area = nda.AreaMembership.fetch(format = 'frame').reset_index()

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [8]:
# Drop cells that were incorrectly extracted and that present duplicated entries
func_data = func_data[func_data['pt_root_id'] != 0]
func_data=func_data.drop_duplicates(subset = 'pt_root_id').iloc[:, 3:]
func_data.head()

,pt_root_id,session,scan_idx,unit_id,pt_position
0,864691136136642173,4,7,665,"[274864, 94064, 22046]"
1,864691136208993724,4,7,740,"[218320, 104320, 19016]"
2,864691135683934135,4,7,782,"[212128, 94544, 23423]"
3,864691135334528233,4,7,855,"[145158, 110036, 19355]"
4,864691136309730650,4,7,921,"[335744, 90128, 22805]"


Layer membership was classified by depth of imaged soma with respect to the dura mater in the structural two-photon stack (Ding et al. 2023): 

L1: 0 − 98 μm; 

L2/3: 98 − 283 μm; 

L4: 283 − 371 μm; 

L5: 371 − 574 μm; 

L6: 574 − 713 μm

In [9]:
#Apply the transform and save it in a new column
func_data['pial_distances'] = tform_vx.apply(func_data['pt_position'])

#Use the y axis value to assign the corresponding layer as per Ding et al. 2023
layers = []
for i in func_data['pial_distances'].iloc[:]:
    if 0<i[1]<=98:
        layers.append('L1')
    elif 98<i[1]<=283:
        layers.append('L2/3')
    elif 283<i[1]<=371:
        layers.append('L4')
    elif 371<i[1]<=574:
        layers.append('L5')
    elif 574<i[1]<=713:
        layers.append('L6')

func_data['cortex_layer'] = layers    

In [10]:
func_data.head()

,pt_root_id,session,scan_idx,unit_id,pt_position,pial_distances,cortex_layer
0,864691136136642173,4,7,665,"[274864, 94064, 22046]","[1062.4793668418956, 73.97710441329255, 881.84...",L1
1,864691136208993724,4,7,740,"[218320, 104320, 19016]","[833.5885576158169, 95.13245843611396, 760.640...",L1
2,864691135683934135,4,7,782,"[212128, 94544, 23423]","[812.3229454978849, 54.01858752556035, 936.920...",L1
3,864691135334528233,4,7,855,"[145158, 110036, 19355]","[540.0614427064812, 92.40350020966703, 774.200...",L1
4,864691136309730650,4,7,921,"[335744, 90128, 22805]","[1306.4448797350165, 79.51718156044586, 912.20...",L1


In [11]:
#Subset only columns of interest
func_data_pos = func_data.copy()
func_data = func_data.iloc[:, [0,1,2,3,6]]

#Add information on the brain area
func_data = func_data.merge(cortex_area, on = ['session', 'scan_idx', 'unit_id'], how = 'left')
func_data_pos = func_data_pos.merge(cortex_area, on = ['session', 'scan_idx', 'unit_id'], how = 'left')

In [12]:
#Save the data
#func_data.to_pickle('clean_functional_neurons.pkl')
func_data_pos.to_pickle('clean_functional_locations.pkl')

In [50]:
func_complete = pd.read_pickle('../analysis_v2/Data/clean_functional_neurons.pkl')

In [70]:
#Extracting only cells from layer 2,3,4
func_l234 = func_complete[(func_complete['cortex_layer'] == 'L2/3') | (func_complete['cortex_layer'] == 'L4')]

In [71]:
#func_l234.to_csv('func_neuronsl234.csv', index = False)